# Analyzing Quantum Volume Errors
This notebook analyzes the error rates required for achieving Quantum Volume at a particular depths. For a given m = depth = number of qubits, plot the HOG for np.logspace outputs to view when it crosses the 2/3rds probability threshold.

In [ ]:
# Configuration parameters. Feel free to mess with these!

import cirq

num_repetitions = 10
depth = 4
num_samplers = 50
device = cirq.google.Bristlecone
compiler = lambda circuit: cirq.google.optimized_for_xmon(
    circuit=circuit,
    new_device=device)

print(f"Configuration: depth {depth} with "
      f"{num_repetitions} runs of {num_samplers} samplers")

In [ ]:
# Run the Quantum Volume algorithm over the above parameters.

import numpy as np
from cirq.contrib import quantum_volume

errors = np.logspace(-1, -4, num=num_samplers)
samplers = [
        cirq.DensityMatrixSimulator(noise=cirq.ConstantQubitNoiseModel(
        qubit_noise_gate=cirq.DepolarizingChannel(p=error)))
    for error in errors]

result = quantum_volume.calculate_quantum_volume(num_repetitions=num_repetitions,
                            depth=depth,
                            num_qubits=depth,
                            device=device,
                            samplers=samplers,
                            compiler=compiler,
                            seed=None)

In [ ]:
# Create a chart that plots the HOG rate relative to the simulated error ratio.

from matplotlib import pyplot as plt

fig, axs = plt.subplots()
sampler_results = [res.sampler_result for res in result] 
axs.plot(errors,
         [sum(res) / len(res) for res in np.transpose(sampler_results)])
         #[sum(res) / len(res) for res in result.sampler_results.values()])

# Line markers for asymptotic ideal heavy output probability and the ideal Heavy
# Output Generation threshold.
axs.axhline((1 + np.log(2)) / 2,
            color='tab:green',
            label='Asymptotic ideal',
            linestyle='dashed')
axs.axhline(2 / 3, label='HOG threshold', color='k', linestyle='dotted')
plt.xscale('log')
axs.set_ybound(0.4, 1)
axs.set_xlabel("error rate")
axs.set_ylabel("est. heavy output probability")
fig.suptitle(f'HOG probability by simulated error rate for d={depth}')